In [1]:
#######################################################################################################################################
######### CRIADO POR      :  MIRIAM OLIVEIRA, LAURO MIRANDA E RAFAEL OLIVEIRA                                                 #########
######### CÓDIGO ADAPTADO :  https://colab.research.google.com/drive/1GXmk1v8C-c4UtyKFqIm9HnsrVYH0pI-c                        #########
######### OBJETIVO      :  EFETUAR A LEITURA DE UMA LISTA DE LIGANTES QUE SERÃO TESTADOS PARA O RECEPTOR 5U1L                 #########
#########                  UTILIZANDO A BIBLIOTECA GNINA. OS DADOS SALVOS SERÃO EXPORTADOS PARA UM ARQUIVO DE SAÍDA           #########
######### DATA          :  24/08/2021                                                                                         #########
######### ARQUIVO DE ENTRADA : LIGANTES.XLSX       (CLICAR COM O BOTÃO DIREITO EM FILES E FAZER UPLOAD DESTE ARQUIVO)         #########
######### ARQUIVO DE SAÍDA   : GNINA_SCORE_RECEPTOR_LIGANTES.csv  (SERÁ SALVO AUTOMATICAMENTE E DISPONIBLIZADO POR DOWNLOAD) #########
###############################################################################################################V#######################

# Instala Miniconda com suporte para Python 3.7
!wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
!bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local

# Cria um novo ambiente Conda com Python 3.7
!conda create -n myenv python=3.7 numpy=1.21 rdkit -y

# Ativa o ambiente Conda
!source activate myenv

# Instala as bibliotecas necessárias
!pip install numpy==1.21.6
!pip install openpyxl
!pip install pandas==1.3.5
!pip install rdkit-pypi #biblioteca usada para manipular e gerar moleculas 3d
!pip install xlrd

# Importa as bibliotecas
import xlrd           #biblioteca usada para manipular arquivo excell
from openpyxl import load_workbook
import pandas as pd   #biblioteca usada para ler os resultados
import subprocess, io, re  #bibliotecas para extração dos resultados do arquivo txt por palavras chave (que contém o score)
from rdkit import Chem #biblioteca usada para manipular e gerar moleculas 3d
from rdkit.Chem import AllChem #biblioteca usada para manipular e gerar moleculas 3d
import os
import numpy

# Instala openbabel usando conda
!conda install -q -y -c conda-forge openbabel

# Adiciona o novo Python ao PATH
import sys
print(f"Python version: {sys.version}")
print(f"NumPy version: {numpy.__version__}")
sys.path.append('/usr/local/lib/python3.7/site-packages/')

# Instala py3Dmol
!pip install py3Dmol

# Baixa smina e gnina
!wget https://downloads.sourceforge.net/project/smina/smina.static
!wget https://github.com/gnina/gnina/releases/download/v1.0.1/gnina
!chmod +x gnina

# Executa gnina
!./gnina

--2024-08-21 18:39:12--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.32.241, 104.16.191.158, 2606:4700::6810:bf9e, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.32.241|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M  10.6MB/s    in 7.5s    

2024-08-21 18:39:20 (10.8 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
./Miniconda3-py37_4.8.2-Linux-x86_64.sh: line 392: /usr/local/conda.exe: Permission denied
chmod: cannot access '/usr/local/conda.exe': No such file or directory
Unpacking payload ...
./Miniconda3-py37_4.8.2-Linux-x86_64.sh: line 404: /usr/local/conda.exe: No such file or directory
./Miniconda3-py37_4.8.2-Linux-x86_64.sh: line 406: /usr/local/conda.exe: No such fil

In [2]:
#LEITURA DO ARQUIVO LIGANTES.XLSX  (MUDE O NOME DO ARQUIVO NESTE PONTO!)

ArqListaLigantes='./Planilha_Ambinter.xls'  # para executar tudo de uma vez

# Altere o 0 para a linha onde os dados começam a ser exibidos
tabela = xlrd.open_workbook(ArqListaLigantes).sheet_by_index(0)
qtd_linhas = tabela.nrows

print("TOTAL DE LIGANTES: " + str(qtd_linhas))
print("\n")

data = []

TOTAL DE LIGANTES: 100




In [ ]:
data = []

try:
    total_molecules = qtd_linhas  # Total de moléculas a serem processadas

    for linha in range(0, total_molecules):
        try:
            # Ler as colunas da tabela
            grupoMolecula = tabela.row(linha)[0].value.strip()  # Ler coluna GRUPO
            nomeMolecula = tabela.row(linha)[1].value.strip()  # Ler coluna MOLECULA
            smileMolecula = tabela.row(linha)[2].value.strip()  # Ler coluna SMILE

            print("=======================================INÍCIO DO PROCESSAMENTO===========================================")
            print(f"\nGRUPO: {grupoMolecula}\nMOLECULA: {nomeMolecula}\nSMILE: {smileMolecula}\n")

            # Gravar arquivo SMILE
            with open("smile.txt", "w") as arquivo:
                arquivo.write(smileMolecula)

            # Converter SMILES para objeto de molécula RDKit
            molecule = Chem.MolFromSmiles(smileMolecula)
            molecule = Chem.AddHs(molecule)
            AllChem.EmbedMolecule(molecule, AllChem.ETKDG())
            AllChem.UFFOptimizeMolecule(molecule, maxIters=100)
            with open('molecule.sdf', 'a') as f:
                f.write(Chem.MolToMolBlock(molecule))

            # Executar GNINA
            os.system('./gnina -r ./Maestro_hp2x7p_alphafold_trimerico.pdb -l molecule.sdf --center_x -7.218 --center_y -44.877 --center_z -2.503 --size_x 50 --size_y 40 --size_z 40 --seed -1985 > scores.txt 2>&1')

            # Ler e processar os resultados
            with open("scores.txt", "r") as file:
                lines = file.readlines()
            for line in lines:
                line = line.strip()
                if re.match(r"\s*\d+\s+[-+]?\d+\.\d+\s+[-+]?\d+\.\d+\s+[-+]?\d+\.\d+", line):
                    columns = line.split()
                    mode = int(columns[0])
                    affinity = float(columns[1])
                    cnn_score = float(columns[2])
                    cnn_affinity = float(columns[3])
                    data.append((mode, affinity, cnn_score, cnn_affinity, "Success", smileMolecula, grupoMolecula, nomeMolecula))

        except Exception as e:
            print(f"Erro ao processar a molécula {nomeMolecula}: {e}")
            data.append(("N/A", "N/A", "N/A", "N/A", "Error", smileMolecula, grupoMolecula, nomeMolecula))

        # Imprime quantas moléculas faltam
        remaining_molecules = total_molecules - (linha + 1)
        print(f"Moléculas restantes: {remaining_molecules}")

except Exception as e:
    print(f"Erro geral: {e}")

# Converter os dados em um DataFrame
scores_df = pd.DataFrame(data, columns=['Mode', 'Affinity', 'CNN Score', 'CNN Affinity', 'Status', "Smile", "Molecule Group", "Molecule Name"])

# Converter DataFrame em Excel
scores_df.to_excel('scores.xlsx', index=False)

=======================================INÍCIO DO PROCESSAMENTO===========================================

GRUPO: Amb10069245
MOLECULA: ZINC000001690027
SMILE: CC(=O)c1cccc(C)n1

Moléculas restantes: 99
=======================================INÍCIO DO PROCESSAMENTO===========================================

GRUPO: Amb10069355
MOLECULA: ZINC000001549593
SMILE: O=C(O)[C@H](O)[C@H](O)[C@@H](O)[C@@H](O)C(=O)O

Moléculas restantes: 98
=======================================INÍCIO DO PROCESSAMENTO===========================================

GRUPO: Amb10069355
MOLECULA: ZINC000003581021
SMILE: O=C(O)[C@H](O)[C@@H](O)[C@@H](O)[C@@H](O)C(=O)O

Moléculas restantes: 97
=======================================INÍCIO DO PROCESSAMENTO===========================================

GRUPO: Amb10069355
MOLECULA: ZINC000006072527
SMILE: O=C(O)[C@@H](O)[C@@H](O)[C@@H](O)[C@@H](O)C(=O)O

